<a href="https://colab.research.google.com/github/afungue/DataExtraction/blob/main/f1-consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OpenF1 https://openf1.org/
OpenF1 es una API gratuita y de código abierto que proporciona datos en tiempo real e históricos de la Fórmula 1.

La API ofrece una gran cantidad de información, incluyendo tiempos por vuelta, telemetría de los autos, comunicaciones por radio, y más. Ya sea que estés buscando crear paneles interactivos, profundizar en el análisis de las carreras o incluso desarrollar objetos conectados que se iluminen cada vez que tu piloto favorito tome la delantera, OpenF1 lo hace posible.

Los datos se pueden acceder en formatos JSON o CSV, lo que la hace amigable tanto para desarrolladores como para personas sin experiencia técnica. Para comenzar rápidamente, puedes acceder a la API a través de tu navegador web. Se proporciona una URL de ejemplo para cada método como referencia.

In [ ]:
!pip install requests


# Sesiones
Las sesiones disponibles van desde la 0 hasta la 202 (Austin Texas 2024)

In [38]:
from urllib.request import urlopen, Request
import json
import os

# Define la URL de la API para obtener todas las sesiones
url = 'https://api.openf1.org/v1/sessions'
api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

# Configura la solicitud con la clave API en los encabezados
request = Request(url)
request.add_header('Authorization', f'Bearer {api_key}')

try:
    # Realiza la solicitud a la API
    response = urlopen(request)
    data = response.read().decode('utf-8')

    # Cargar la respuesta como JSON
    data_sessions = json.loads(data)

    # Ordenar las sesiones por año (descendente) y date_start (descendente)
    sorted_sessions = sorted(data_sessions, key=lambda x: (x['year'], x['date_start']), reverse=True)

    # Imprimir la información de todas las sesiones en el formato solicitado
    last_year = None
    last_location = None

    for session in sorted_sessions:  # Iterar sobre las sesiones ya ordenadas
        if (session['year'], session['location']) != (last_year, last_location):
            # Solo imprime el año y la localidad una vez
            date_start = session['date_start']
            date_end = session['date_end']
            print(f"{session['year']}, {session['location']} (Inicio: {date_start}, Fin: {date_end})")
            last_year = session['year']
            last_location = session['location']

        # Indentación para las sesiones
        print(f"    {session['session_name']} (ID: {session['session_key']})")

except Exception as e:
    print("An error occurred:", str(e))


2024, Austin (Inicio: 2024-10-18T17:30:00+00:00, Fin: 2024-10-18T18:30:00+00:00)
    Practice 1 (ID: 9607)
2024, Marina Bay (Inicio: 2024-09-22T12:00:00+00:00, Fin: 2024-09-22T14:00:00+00:00)
    Race (ID: 9606)
    Qualifying (ID: 9602)
    Practice 3 (ID: 9601)
    Practice 2 (ID: 9600)
    Practice 1 (ID: 9599)
2024, Baku (Inicio: 2024-09-15T11:00:00+00:00, Fin: 2024-09-15T13:00:00+00:00)
    Race (ID: 9598)
    Qualifying (ID: 9594)
    Practice 3 (ID: 9593)
    Practice 2 (ID: 9592)
    Practice 1 (ID: 9591)
2024, Monza (Inicio: 2024-09-01T13:00:00+00:00, Fin: 2024-09-01T15:00:00+00:00)
    Race (ID: 9590)
    Qualifying (ID: 9586)
    Practice 3 (ID: 9585)
    Practice 2 (ID: 9584)
    Practice 1 (ID: 9583)
2024, Zandvoort (Inicio: 2024-08-25T13:00:00+00:00, Fin: 2024-08-25T15:00:00+00:00)
    Race (ID: 9582)
    Qualifying (ID: 9578)
    Practice 3 (ID: 9577)
    Practice 2 (ID: 9576)
    Practice 1 (ID: 9575)
2024, Spa-Francorchamps (Inicio: 2024-07-28T13:00:00+00:00, Fin: 2024

Ordenamiento de las sesiones por orden cronologico

# Sesiones por año


In [54]:
from urllib.request import urlopen, Request
import json
import os
from datetime import datetime
import pytz  # Importar la librería pytz

# Función para obtener sesiones por año
def get_sessions_by_year(year):
    # Define la URL de la API para obtener todas las sesiones
    url = f'https://api.openf1.org/v1/sessions?year={year}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = response.read().decode('utf-8')

        # Cargar la respuesta como JSON
        data_sessions = json.loads(data)

        # Ordenar las sesiones por date_start de más nueva a más antigua
        data_sessions.sort(key=lambda x: x['date_start'], reverse=True)

        # Imprimir la información de las sesiones
        current_year = None
        current_location = None
        print(f"{'Año':<6} {'Localidad':<20} {'Nombre':<30} {'ID':<6} {'Fecha inicio':<25} {'Fecha fin':<25}")
        print("=" * 120)

        # Definir la zona horaria de Argentina (GMT-3)
        tz = pytz.timezone('America/Argentina/Buenos_Aires')

        for session in data_sessions:
            if session['year'] != current_year:
                current_year = session['year']
                print(f"{current_year:<6}")

            if session['location'] != current_location:
                current_location = session['location']
                print(f"  {current_location:<20}")

            # Convertir las fechas a tu zona horaria
            date_start = datetime.fromisoformat(session['date_start'].replace("Z", "+00:00"))
            date_end = datetime.fromisoformat(session['date_end'].replace("Z", "+00:00"))

            # Ajustar a la zona horaria GMT-3
            date_start = date_start.astimezone(tz)
            date_end = date_end.astimezone(tz)

            # Formatear las fechas
            formatted_start = date_start.strftime("%d %B %Y, %H:%M")
            formatted_end = date_end.strftime("%d %B %Y, %H:%M")

            # Imprimir nombre de sesión, ID y fechas con ajuste de espacio
            print(f"    {session['session_name']:<18} (ID: {session['session_key']:<5}) - Fecha inicio: {formatted_start:<24} - Fecha fin: {formatted_end:<24}")

    except Exception as e:
        print("An error occurred:", str(e))

# Ejemplo de uso
get_sessions_by_year(2024)  # Cambia el año según lo necesites


Año    Localidad            Nombre                         ID     Fecha inicio              Fecha fin                
2024  
  Austin              
    Practice 1         (ID: 9607 ) - Fecha inicio: 18 October 2024, 14:30   - Fecha fin: 18 October 2024, 15:30  
  Marina Bay          
    Race               (ID: 9606 ) - Fecha inicio: 22 September 2024, 09:00 - Fecha fin: 22 September 2024, 11:00
    Qualifying         (ID: 9602 ) - Fecha inicio: 21 September 2024, 10:00 - Fecha fin: 21 September 2024, 11:00
    Practice 3         (ID: 9601 ) - Fecha inicio: 21 September 2024, 06:30 - Fecha fin: 21 September 2024, 07:30
    Practice 2         (ID: 9600 ) - Fecha inicio: 20 September 2024, 10:00 - Fecha fin: 20 September 2024, 11:00
    Practice 1         (ID: 9599 ) - Fecha inicio: 20 September 2024, 06:30 - Fecha fin: 20 September 2024, 07:30
  Baku                
    Race               (ID: 9598 ) - Fecha inicio: 15 September 2024, 08:00 - Fecha fin: 15 September 2024, 10:00
    Qual

In [21]:
from urllib.request import urlopen, Request
import json
import os

# Define la URL de la API y tu clave API
session_key = 202
url = f'https://api.openf1.org/v1/sessions?session_key={session_key}'
api_key = os.getenv('OPENF1')  # Asegúrate de que la clave API esté en la variable de entorno

# Configura la solicitud con la clave API en los encabezados
request = Request(url)
request.add_header('Authorization', f'Bearer {api_key}')

try:
    # Realiza la solicitud a la API
    response = urlopen(request)
    data = response.read().decode('utf-8')

    # Cargar la respuesta como JSON
    data_sessions = json.loads(data)

    # Imprimir la información de la sesión
    for session in data_sessions:
        print(f"Session Name: {session['session_name']}, Year: {session['year']}, Location: {session['location']}")

except Exception as e:
    print("An error occurred:", str(e))


Informacion de un piloto en particular

In [6]:
import requests
import os

# Define la URL de la API
url = "https://api.openf1.org/v1/car_data"

# Define los parámetros de la consulta
params = {
    'driver_number': 55,  # Cambia este número según el conductor que te interese
    'session_key': 9159,  # Cambia esto por la clave de sesión que necesites
    'speed>=': 315  # Este es el filtro para la velocidad mínima
}

# Realiza la solicitud GET
response = requests.get(url, params=params)

# Verifica si la solicitud fue exitosa
if response.status_code == 200:
    # Procesa los datos JSON
    data = response.json()

    # Imprime la respuesta completa para inspección
    print("Response JSON:", data)

    # Opcional: importar los resultados en un DataFrame (necesitas instalar el paquete `pandas` primero)
    try:
        import pandas as pd

        df = pd.DataFrame(data)
        print("DataFrame:")
        print(df)
    except ImportError:
        print("Pandas no está instalado. Instala pandas para usar esta funcionalidad.")
else:
    print(f"Error: {response.status_code}, {response.text}")


Response JSON: []
DataFrame:
Empty DataFrame
Columns: []
Index: []
